In [30]:
import numpy as np
import pandas as pd
import random
import time
from itertools import product
from scipy.stats import norm, binomtest
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Lasso, Ridge

from sklearn import linear_model, svm, naive_bayes, ensemble
from sklearn.model_selection import cross_validate, train_test_split, RepeatedStratifiedKFold
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, roc_auc_score, matthews_corrcoef

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


## Read functions

In [31]:
%run functions_scenario2.ipynb

## Scenario 2, Case 2 with interaction

In [32]:
def invlogit(p_list):
    invlogit_values = []
    for p in p_list:
        invlogit_values.append(np.exp(p) / (1 + np.exp(p)))

    return invlogit_values

In [33]:
design_list = [5,8,10]
patient_n_list = [5400,680,170] # n_patient_per_plan

# def plan_true_rate(row, impact_rates):
#     tmp = row * impact_rates
#     tmp = tmp[tmp != 0]
#     return np.prod(tmp) * 0.05

def generate_data_step1(design_number = 5, n_rounds = 4, n_patient_per_plan = 5400, seed=42):

  # design_list = [5,8,10]

  # Record the start time
#   start_time = time.time()

  # scenario 2:
  random.seed(seed)

  # n_design = design_number
  n_patient_per_plan_round = n_patient_per_plan/n_rounds # each round is capped by this patient number

  # Create all possible combinations of design features
  design_combinations = list(product([1, 0], repeat=n_design))

  design_feature_df = pd.DataFrame(design_combinations, columns=[f'Design_Feature_{i+1}' for i in range(n_design)])
  # add interaction term:
  interaction = [1 if row['Design_Feature_1'] + row['Design_Feature_2'] == 2 else 0 for _, row in design_feature_df.iterrows()]
  design_feature_df['interaction'] = interaction

  design_feature_df['recruitment_plan'] = [x + 1 for x in range(2 ** n_design)]

  # design_feature_df
  random.seed(seed)
  # impact_rates = np.array([random.uniform(0.6, 1.5) for _ in range(n_design)])
  impact_rates = np.array([-0.5, 0, 0, 0, 0, 0, 0, 0, 1]) # design 2 has the highest rr
  response_rate_list = np.dot(np.array(design_feature_df.iloc[:,:(n_design+1)]), impact_rates)
  # normalize to [0,1]
  # normalized_values = normalize_to_0_1(response_rate_list) * 0.115
  # inverse logit
  normalized_values = np.array(invlogit(response_rate_list)) * 0.11
  design_feature_df['plan_response_rate'] = normalized_values

  design_feature_df.drop(columns=['interaction'], axis = 1, inplace=True)

  # Apply plan_true_rate to each row
  # result_column = design_feature_df.iloc[:, :n_design].apply(lambda row: plan_true_rate(row, impact_rates), axis=1)

  # Add the result_column to the design_feature_df
  # design_feature_df['plan_response_rate'] = result_column

  # each combination repeat for n_patient_per_plan_round
  design_feature_df = pd.DataFrame(np.repeat(design_feature_df.values, n_patient_per_plan_round, axis=0), columns=design_feature_df.columns)

  # add response outcome column:
  grouped_df = design_feature_df.groupby(list(design_feature_df.columns)).size().reset_index(name='group_size')

  random.seed(seed)
  random_seeds_for_resp = [random.randint(1, 100000) for _ in range(len(grouped_df))]

  def generate_responses(row):
      rate = row['plan_response_rate']
      num = row['group_size']
      # random.seed(42)
      seed1 = random_seeds_for_resp[row.name]
      np.random.seed(seed1)
      return np.random.binomial(n=1, p=rate, size=int(num))

  grouped_df['response'] = grouped_df.apply(generate_responses, axis=1)

  # Explode the 'response' arrays to expand the DataFrame
  expanded_df = grouped_df.explode('response').reset_index(drop=True)
  expanded_df['response'] = expanded_df['response'].astype(float)

#   end_time = time.time()

#   elapsed_time = end_time - start_time

#   print(f"Elapsed time: {elapsed_time} seconds")

  return expanded_df

## Logistic regression

In [115]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0))
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
#                     ('RF', ensemble.RandomForestClassifier(n_estimators=200, criterion='gini', random_state=0))
                    # ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        'LR__C': [0.01, 0.1, 1.0] # [0.01, 0.05, 0.1]
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
#         'RF__n_estimators': [50] # [10, 30, 50]
        # 'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print(best_params)

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [116]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [117]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 8
n_patient_per_plan = 680
n_rounds = 6
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [118]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []
highest_true_rr = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    highest_true_rr.append(np.max(dt_design_5['plan_response_rate']))
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



1
['step1_rr: 0', 'step2_rr: 0', 'step3_rr: 0', 'step4_rr: 0', 'step5_rr: 0', 'step6_rr: 0']
empirical response rate 0.05295907079646018
{'LR__C': 1.0}
remaining size for Round 2: 145152
Calculated size for Round 2: 4085.324125976102
{'LR__C': 0.1}
orr termination 0.016484381840233098 0.08190069999881822 0.06541631815858512
remaining size for Round 3: 141067
Calculated size for Round 3: 11720.937161727807
{'LR__C': 0.1}
orr termination 0.017755218243390578 0.07305221580509873 0.05529699756170815
remaining size for Round 4: 129347
2
['step1_rr: 1', 'step2_rr: 1', 'step3_rr: 1', 'step4_rr: 1', 'step5_rr: 1', 'step6_rr: 1']
empirical response rate 0.056312223451327435
{'LR__C': 0.01}
remaining size for Round 2: 145152
Calculated size for Round 2: 4318.382983229351
{'LR__C': 0.01}
orr termination 0.05126019033147661 0.06560612306027376 0.014345932728797153
remaining size for Round 3: 140834
Calculated size for Round 3: 5093.458360806866
{'LR__C': 1.0}
orr termination 0.03260075976847697 0.

### Results summary

In [119]:
rr_df = pd.DataFrame(rr_dict)
rr_df

,step1_rr,step2_rr,step3_rr,step4_rr,step5_rr,step6_rr
0,0.055,0.064535,0.068471,0.068471,NaN,NaN
1,0.055,0.061733,0.064146,0.068471,0.068471,0.068471
2,0.055,0.061287,0.065467,0.065890,0.068471,0.068471
3,0.055,0.064290,0.067729,0.068471,0.068471,0.068471
4,0.055,0.061436,0.068471,0.068471,0.068471,0.068471
...,...,...,...,...,...,...
95,0.055,0.061533,0.065446,0.068471,0.068471,0.068471
96,0.055,0.061175,0.065854,0.068471,0.068471,0.068471
97,0.055,0.061737,0.068471,0.068471,NaN,NaN
98,0.055,0.061193,0.068471,0.068471,0.068471,0.068471


In [120]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

5.5 (0.8)


In [121]:
# orr for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_rr: 0.055 (0.000)
step2_rr: 0.061 (0.002)
step3_rr: 0.066 (0.003)
step4_rr: 0.067 (0.003)
step5_rr: 0.068 (0.002)
step6_rr: 0.068 (0.002)


In [122]:
# overall orr:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

Mean: 0.066, StD: 0.002


In [123]:
# better chance
np.mean([binomtest(int(x), n=total_n, p=0.055).pvalue < 0.05 for x in orr_total])

0.96

In [124]:
# highest true rr:
mean_result=(np.mean(highest_true_rr))
std_result=(np.std(highest_true_rr))
print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

Mean: 0.068 (0.000)


In [125]:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

Mean: 0.071, StD: 0.003


In [126]:
# average plan number for each round:
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_plan_number: nan (nan)
step2_plan_number: 93.4 (41.0)
step3_plan_number: 43.0 (33.8)
step4_plan_number: 16.2 (18.1)
step5_plan_number: 6.2 (8.3)
step6_plan_number: 2.3 (2.4)


In [127]:
# early stopping probabilities:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

{'early_stopping': 0.31,
 'early_stopping_plan': 0.31,
 'early_stopping_orr': 0.0,
 'early_stopping_size': 0.0}

In [128]:
# sample size for the last round:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

124136.01 4840.649331432715


In [129]:
# sample size for the last round:
np.mean(better_chance)

0.96

In [130]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_sample_size: 28928.000 (0.000)
step2_sample_size: 4217.873 (88.452)
step3_sample_size: 8557.131 (18943.585)
step4_sample_size: 19836.461 (38239.027)
step5_sample_size: 32977.896 (49233.014)
step6_sample_size: 121621.043 (2016.705)


## Random Forest

In [131]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            # ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0))
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
                    ('RF', ensemble.RandomForestClassifier(criterion='gini', random_state=0))
                    # ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        # 'LR__C': [0.01] # [0.01, 0.05, 0.1]
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
        'RF__n_estimators': [50, 100, 200]
#         'RF__max_depth': [10, 20, None]
        # 'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print(best_params)

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [132]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [133]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 8
n_patient_per_plan = 680
n_rounds = 6
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [134]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



1
['step1_rr: 0', 'step2_rr: 0', 'step3_rr: 0', 'step4_rr: 0', 'step5_rr: 0', 'step6_rr: 0']
empirical response rate 0.05295907079646018
{'RF__n_estimators': 50}
remaining size for Round 2: 145152
Calculated size for Round 2: 4085.324125976102
{'RF__n_estimators': 100}
orr termination 0.048293184326750345 0.08368479062356582 0.035391606296815474
remaining size for Round 3: 141067
Calculated size for Round 3: 9881.299459027116
{'RF__n_estimators': 100}
orr termination 0.03583852627335993 0.07703158419021709 0.041193057916857165
remaining size for Round 4: 131186
Calculated size for Round 4: 6235.480974022526
{'RF__n_estimators': 200}
orr termination 0.018977860891060884 0.07944314900211277 0.06046528811105189
remaining size for Round 5: 124951
Calculated size for Round 5: 6710.104801605472
{'RF__n_estimators': 50}
orr termination 0.03893929786771238 0.07661048378864778 0.0376711859209354
remaining size for last Round 6: 118241
2
['step1_rr: 1', 'step2_rr: 1', 'step3_rr: 1', 'step4_rr: 1

### Results summary

In [135]:
rr_df = pd.DataFrame(rr_dict)
rr_df

,step1_rr,step2_rr,step3_rr,step4_rr,step5_rr,step6_rr
0,0.055,0.061640,0.062863,0.064313,0.068471,0.068471
1,0.055,0.058775,0.065518,0.066788,0.068471,0.068471
2,0.055,0.057087,0.061874,0.064323,0.068471,0.068471
3,0.055,0.060335,0.061994,0.065077,0.068471,NaN
4,0.055,0.057650,0.060181,0.068471,NaN,NaN
...,...,...,...,...,...,...
95,0.055,0.060020,0.063125,0.068471,0.068471,0.068471
96,0.055,0.061341,0.067013,0.068471,0.068471,NaN
97,0.055,0.059805,0.062906,0.065931,0.068471,0.068471
98,0.055,0.058230,0.060300,0.059651,0.062180,0.068471


In [136]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

5.4 (0.7)


In [137]:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_rr: 0.055 (0.000)
step2_rr: 0.060 (0.002)
step3_rr: 0.063 (0.003)
step4_rr: 0.066 (0.002)
step5_rr: 0.068 (0.001)
step6_rr: 0.068 (0.000)


In [138]:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

Mean: 0.066 (0.000)


In [139]:
mean_result=(np.mean(highest_true_rr))
std_result=(np.std(highest_true_rr))
print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

Mean: 0.068 (0.000)


In [140]:
result_dict = np.array(orr_total_adaptive) / (145152)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f} ({std_result:.3f})")

Mean: 0.068 (0.000)


In [141]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_plan_number: nan (nan)
step2_plan_number: 95.9 (59.1)
step3_plan_number: 24.2 (22.6)
step4_plan_number: 7.5 (11.4)
step5_plan_number: 3.1 (2.7)
step6_plan_number: 2.3 (2.1)


In [142]:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

{'early_stopping': 0.46,
 'early_stopping_plan': 0.46,
 'early_stopping_orr': 0.0,
 'early_stopping_size': 0.0}

In [143]:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

122392.93 5521.034930617628


In [144]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_sample_size: 28928.000 (0.000)
step2_sample_size: 4217.873 (88.452)
step3_sample_size: 10760.691 (19115.072)
step4_sample_size: 20053.156 (36831.305)
step5_sample_size: 52410.894 (57130.650)
step6_sample_size: 119261.796 (2096.378)


In [145]:
np.mean(better_chance)

1.0

## XGBoost

In [146]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            # ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0))
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
#                     ('RF', ensemble.RandomForestClassifier(n_estimators=200, criterion='gini', random_state=0))
                    ('XGB', XGBClassifier(n_estimators=50, learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        # 'LR__C': [0.01] # [0.01, 0.05, 0.1]
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.01, 0.05, 0.1]
#         'RF__n_estimators': [50] # [10, 30, 50]
        'XGB__learning_rate': [0.01, 0.1, 1],
        'XGB__n_estimators': [50, 100, 200]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("Best fit parameters:", best_params)

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [147]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [148]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 8
n_patient_per_plan = 680
n_rounds = 6
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [ ]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []
highest_true_rr = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    highest_true_rr.append(np.max(dt_design_5['plan_response_rate']))
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



1
['step1_rr: 0', 'step2_rr: 0', 'step3_rr: 0', 'step4_rr: 0', 'step5_rr: 0', 'step6_rr: 0']
empirical response rate 0.05295907079646018
Best fit parameters: {'XGB__learning_rate': 0.01, 'XGB__n_estimators': 200}
remaining size for Round 2: 145152
Calculated size for Round 2: 4085.324125976102
Best fit parameters: {'XGB__learning_rate': 1, 'XGB__n_estimators': 50}
orr termination 0.0024932992265810283 0.10169291496276855 0.09919961573618753
remaining size for Round 3: 141067
2
['step1_rr: 1', 'step2_rr: 1', 'step3_rr: 1', 'step4_rr: 1', 'step5_rr: 1', 'step6_rr: 1']
empirical response rate 0.056312223451327435
Best fit parameters: {'XGB__learning_rate': 0.01, 'XGB__n_estimators': 50}
remaining size for Round 2: 145152
Calculated size for Round 2: 4318.382983229351
Best fit parameters: {'XGB__learning_rate': 0.01, 'XGB__n_estimators': 200}
orr termination 0.024621968277250744 0.2114297178521184 0.18680774957486765
remaining size for Round 3: 140834
Calculated size for Round 3: 5638.3182

### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
print(np.mean(highest_rr_overall))
print(np.std(highest_rr_overall))

In [ ]:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

## Ensemble learning - 3 methods

In [34]:
# def ensemble_model_fit(data, data_pred):
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the VotingClassifier with the individual classifiers
#     voting_classifier = ensemble.VotingClassifier(
#         estimators=[
#             ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0)),
# #             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
#                     # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
#                     ('RF', ensemble.RandomForestClassifier(criterion='gini', random_state=0)),
#                     ('XGB', XGBClassifier(learning_rate=0.1, random_state=0))
#                    ],
#         voting='soft'
#     )

#     # Define the hyperparameter grid to search
#     # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
#     param_grid = {
#         # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
#         'LR__C': [0.01, 0.05, 0.1],
#         # 'Ridge__C': [0.01]
#         # 'SVM__C': [0.01, 0.05, 0.1]
#         'RF__n_estimators': [50, 100, 200],
#         'XGB__n_estimators': [50, 100, 200]
#     }

#     # Create a GridSearchCV object
#     # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
#     grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

#     # Perform the grid search on the training data
#     grid_search.fit(X_train, y_train)

#     # Get the best hyperparameters
#     best_params = grid_search.best_params_
#     # Print the best fitted parameters
#     print("Best fitted parameters:")
#     print(best_params)

#     # Train the final VotingClassifier with the best hyperparameters on the full training set
#     final_voting_classifier = grid_search.best_estimator_
#     final_voting_classifier.fit(X_train, y_train)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
#     y_pred_test = final_voting_classifier.predict(X_test)
#     X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
#     y_pred = final_voting_classifier.predict_proba(X_dt)

#     return y_pred

In [35]:
def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(max_iter=200, random_state=0)),
#             ('Ridge', linear_model.LogisticRegression(penalty='l2', solver='lbfgs', max_iter=200, random_state=0))
                    # ('SVM', svm.SVC(kernel='linear', C=1.0, random_state=0, probability=True, class_weight='balanced'))
                    ('RF', ensemble.RandomForestClassifier(criterion='gini', random_state=0)),
                    ('XGB', XGBClassifier(learning_rate=0.1, random_state=0))
                   ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
    # Best Hyperparameters: {'LR__C': 0.01, 'RF__n_estimators': 50, 'XGB__n_estimators': 50}
    param_grid = {
        # 'NB__alpha': [0.01, 0.05, 0.1],  # '__' is used to specify hyperparameters for individual classifiers
        'LR__C': [0.05],
        # 'Ridge__C': [0.01]
        # 'SVM__C': [0.05]
        'RF__n_estimators': [100],
        'XGB__n_estimators': [50]
    }

    # Create a GridSearchCV object
    # custom_scorer_auc = make_scorer(roc_auc_score, needs_proba=True)
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan','plan_response_rate','group_size','response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred

In [36]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# def ensemble_model_fit(data, data_pred):
#     # Split the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(
#         data.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1),
#         data['response'],
#         test_size=0.2,
#         random_state=0
#     )

#     # Define the Logistic Regression model
#     logistic_regression = LogisticRegression(C=0.01, max_iter=200,random_state=0)

#     # Fit the Logistic Regression model on the training data
#     logistic_regression.fit(X_train, y_train)

#     # Print out the coefficients of the logistic regression model
#     print("Coefficients:", logistic_regression.coef_)
#     print("Intercept:", logistic_regression.intercept_)
#     print("Coefficients shape:", logistic_regression.coef_.shape)

#     # Predict probabilities instead of binary outcomes on the test set
#     y_pred_proba_test = logistic_regression.predict_proba(X_test)
#     y_pred_test = logistic_regression.predict(X_test)

#     # You can also predict probabilities for the prediction data (data_pred)
#     X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size','response'], axis=1)
#     y_pred = logistic_regression.predict_proba(X_dt)

#     return y_pred

# # Call the function
# # ensemble_model_fit(data, data_pred)


### Simulation starts

In [50]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 8
n_patient_per_plan = 680
n_rounds = 6
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [51]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



1
['step1_rr: 0', 'step2_rr: 0', 'step3_rr: 0', 'step4_rr: 0', 'step5_rr: 0', 'step6_rr: 0']
empirical response rate 0.05295907079646018
remaining size for Round 2: 145152
Calculated size for Round 2: 4085.324125976102
orr termination 0.01463903137611023 0.08982638556159091 0.07518735418548067
remaining size for Round 3: 141067
Calculated size for Round 3: 12223.008371224478
orr termination 0.04002275760035494 0.07963577136267834 0.0396130137623234
remaining size for Round 4: 128844
Calculated size for Round 4: 6247.398449996775
orr termination 0.03923737112962387 0.0783287470927136 0.03909137596308974
remaining size for Round 5: 122597
2
['step1_rr: 1', 'step2_rr: 1', 'step3_rr: 1', 'step4_rr: 1', 'step5_rr: 1', 'step6_rr: 1']
empirical response rate 0.056312223451327435
remaining size for Round 2: 145152
Calculated size for Round 2: 4318.382983229351
orr termination 0.017924446255585827 0.09104622071737775 0.07312177446179192
remaining size for Round 3: 140834
Calculated size for Rou

### Results summary

In [52]:
rr_df = pd.DataFrame(rr_dict)
rr_df

,step1_rr,step2_rr,step3_rr,step4_rr,step5_rr,step6_rr
0,0.055,0.067462,0.065137,0.068471,0.068471,NaN
1,0.055,0.061899,0.065524,0.068471,0.068471,0.068471
2,0.055,0.063692,0.065209,0.068471,0.068471,NaN
3,0.055,0.065569,0.068471,0.068471,0.068471,NaN
4,0.055,0.060871,0.063912,0.068471,0.068471,NaN
...,...,...,...,...,...,...
95,0.055,0.063298,0.065608,0.066469,0.068471,0.068471
96,0.055,0.063416,0.064060,0.065011,0.066209,0.065256
97,0.055,0.061289,0.062022,0.068471,0.068471,NaN
98,0.055,0.061218,0.064074,0.068471,0.068471,NaN


In [53]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

5.2 (0.9)


In [54]:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_rr: 0.055 (0.000)
step2_rr: 0.064 (0.003)
step3_rr: 0.065 (0.003)
step4_rr: 0.067 (0.003)
step5_rr: 0.068 (0.001)
step6_rr: 0.068 (0.001)


In [55]:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

Mean: 0.066, StD: 0.002


In [56]:
print(np.mean(highest_rr_overall))
print(np.std(highest_rr_overall))

0.06806641063923788
0.0022978996707203243


In [57]:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

Mean: 0.071, StD: 0.002


In [58]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_plan_number: nan (nan)
step2_plan_number: 63.2 (42.8)
step3_plan_number: 20.8 (22.8)
step4_plan_number: 6.9 (8.8)
step5_plan_number: 3.4 (3.4)
step6_plan_number: 2.4 (2.1)


In [59]:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

{'early_stopping': 0.54,
 'early_stopping_plan': 0.54,
 'early_stopping_orr': 0.0,
 'early_stopping_size': 0.0}

In [60]:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

123036.39 6263.472333929481


In [61]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

step1_sample_size: 28928.000 (0.000)
step2_sample_size: 4217.873 (88.452)
step3_sample_size: 14793.484 (26112.962)
step4_sample_size: 31024.123 (48303.331)
step5_sample_size: 54188.328 (57322.855)
step6_sample_size: 118769.652 (2391.621)


## Ensemble learning - 7 methods

In [62]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import ensemble, linear_model
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

def ensemble_model_fit(data, data_pred):
    X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['recruitment_plan', 'plan_response_rate', 'group_size', 'response'], axis=1),
        data['response'],
        test_size=0.2,
        random_state=0
    )

    # Define the VotingClassifier with the individual classifiers
    voting_classifier = ensemble.VotingClassifier(
        estimators=[
            ('LR', linear_model.LogisticRegression(penalty = 'none', max_iter=200, random_state=0)),
            ('Lasso', linear_model.LogisticRegression(penalty = "l1", max_iter=200, random_state=0,
                                                     solver="liblinear")),
            ('Ridge', linear_model.LogisticRegression(penalty = "l2", max_iter=200, random_state=0)),
            ('GBM', ensemble.GradientBoostingClassifier(random_state=0)),
            ('RF', ensemble.RandomForestClassifier(random_state=0)),
            ('XGB', XGBClassifier(random_state=0)),
            ('NN', MLPClassifier(random_state=0))
        ],
        voting='soft'
    )

    # Define the hyperparameter grid to search
#     param_grid = {
# #         'LR__C': [0.01, 0.1, 1.0],  # Regularization parameter for logistic regression
#         'Lasso__C': [0.01, 0.1, 1.0],  # Regularization parameter for lasso regression
#         'Ridge__C': [0.01, 0.1, 1.0],  # Regularization parameter for ridge regression
#         'GBM__learning_rate': [0.01, 0.1, 0.5],  # Learning rate for gradient boosting machine
#         'GBM__n_estimators': [50, 100, 200],  # Number of trees for gradient boosting machine
#         'RF__n_estimators': [50, 100, 200],  # Number of trees for random forest
# #         'RF__max_depth': [10, 20],  # Maximum depth of trees for random forest
#         'XGB__learning_rate': [0.01, 0.1, 0.5],  # Learning rate for XGBoost
#         'XGB__n_estimators': [50, 100, 200],  # Number of trees for XGBoost
#         'NN__hidden_layer_sizes': [(50,), (100,), (50, 50)],  # Size of hidden layers for neural networks
#         'NN__alpha': [0.0001, 0.001, 0.01]  # Regularization parameter for neural networks
#     }
    param_grid = {
#         'LR__C': [0.01, 0.1, 1.0],  # Regularization parameter for logistic regression
        'Lasso__C': [0.1],  # Regularization parameter for lasso regression
        'Ridge__C': [0.01],  # Regularization parameter for ridge regression
        'GBM__learning_rate': [0.01],  # Learning rate for gradient boosting machine
        'GBM__n_estimators': [50],  # Number of trees for gradient boosting machine
        'RF__n_estimators': [50],  # Number of trees for random forest
#         'RF__max_depth': [10, 20],  # Maximum depth of trees for random forest
        'XGB__learning_rate': [0.01],  # Learning rate for XGBoost
        'XGB__n_estimators': [50],  # Number of trees for XGBoost
        'NN__hidden_layer_sizes': [(50,)],  # Size of hidden layers for neural networks
        'NN__alpha': [0.01]  # Regularization parameter for neural networks
    }

    # Create a GridSearchCV object
    grid_search = GridSearchCV(voting_classifier, param_grid, cv=10, scoring='roc_auc')

    # Perform the grid search on the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
#     print("Best fit parameters:", best_params)

    # Train the final VotingClassifier with the best hyperparameters on the full training set
    final_voting_classifier = grid_search.best_estimator_
    final_voting_classifier.fit(X_train, y_train)

    # Predict probabilities instead of binary outcomes on the test set
    y_pred_proba_test = final_voting_classifier.predict_proba(X_test)
    y_pred_test = final_voting_classifier.predict(X_test)
    X_dt = data_pred.drop(['recruitment_plan', 'plan_response_rate', 'group_size', 'response'], axis=1)
    y_pred = final_voting_classifier.predict_proba(X_dt)

    return y_pred


In [63]:
# ensemble_model_fit(dt_design_5, dt_design_5)

### Simulation starts

In [64]:
n_sim = 100
# create a list of random seeds
random.seed(42)
random_seeds = [random.randint(1, 100000) for _ in range(n_sim)]

design_list = [5,8,10]
patient_n_list = [5468,680,170] # n_patient_per_plan

n_design = 6
n_patient_per_plan = 680
n_rounds = 8
total_n = n_patient_per_plan * (2**n_design)

## sample size determination
beta = 0.2
power = 1 - beta
alpha = 0.05
delta = 0.01 # effect size

## early stopping
epsilon = 0.001

In [65]:
rr_dict = {"step{}_rr".format(r): [] for r in range(1, n_rounds + 1)}
plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
sample_size_dict = {"step{}_sample_size".format(r): [] for r in range(1, n_rounds + 1)}
last_round_sample_size = []
random_rr_dict = {"step{}_random_max_rr".format(r): [] for r in range(1, n_rounds + 1)}
random_rr_dict.update({
    "step{}_random_mean_rr".format(r): [] for r in range(1, n_rounds + 1)
})

stopping_dict = {"early_stopping": [],
                 "early_stopping_plan":[],
                 "early_stopping_orr":[],
                 "early_stopping_size":[]}
final_plan_number = []
highest_rr_overall = []
highest_true_rr = []

better_chance = []

orr_total = []
orr_total_adaptive = []



i = 0

for seed in random_seeds:
    i += 1
    print(i)

    print([f"{key}: {len(value)}" for key, value in rr_dict.items()])



    event_list = 0
    event_list_adaptive = 0
    max_rr = []

    # step 1:
    ## Generate dataset
    dt_design_5 = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed)
    highest_true_rr.append(np.max(dt_design_5['plan_response_rate']))
    print("empirical response rate", dt_design_5['response'].mean())
    ## save for benchmark results
    maxidx = np.argmax(dt_design_5['plan_response_rate'])
    random_rr_dict['step1_random_max_rr'].append(dt_design_5.iloc[maxidx,6])
    random_rr_dict['step1_random_mean_rr'].append(dt_design_5['plan_response_rate'].mean())

    ## Ensemble modelling:
    y_pred = ensemble_model_fit(data=dt_design_5, data_pred = dt_design_5)

    ## select recruitment plan
    pred_df = pd.DataFrame(np.hstack((dt_design_5,  y_pred[:, 1].reshape(-1, 1))),
                             columns=list(dt_design_5.columns) + ['predicted_response_rate'])
    pred_df_rr = pred_df.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

    x = pred_df_rr['predicted_response_rate'].values
    if len(x) <= 10:
        best_k = 2
    else:
        best_k = kmeans_fit(data = x)['best_k']
    kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

    merged_df = pd.merge(pred_df_rr, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
    cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
    highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
    highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)


    p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))
    highest_cluster['p_vec'] = p_vec_next
    highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

    # highest_cluster = pred_df_rr

    # highest_cluster = pd.merge(highest_cluster, dt_design_5[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')
    # highest_cluster['cluster_number'] = highest_cluster['recruitment_plan']

    ## prepare to chance of better performance:
    temp = dt_design_5[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
    event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print("step1", np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
    # print(temp)
    p0 = (temp['plan_response_rate'].mean())

    rr_dict["step1_rr"].append(dt_design_5['plan_response_rate'].mean())
    sample_size_dict["step1_sample_size"].append(len(dt_design_5))

    for round in range(2, n_rounds+1):

        rr_dict["step"+str(round)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))
        plan_number_dict["step"+str(round)+"_plan_number"].append(len(p_vec_next))

        ## when it comes to the last round:
        if round == n_rounds:

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            if round == 2: # if the last round is round 2
                remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
            else:
                remaining_size -= len(dt_design_5_step2up)

            print("remaining size for last Round " + str(round) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            ## data generation, combine previous data:
            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                          design_number = n_design, n_rounds = n_rounds,
                                                          n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(0)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(len(temp), temp['plan_response_rate'].mean())
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



        ## If haven't reached the last round:
        ## check remaining sample size:
        if round == 2:
            remaining_size = total_n - len(dt_design_5) # apply to the rest of the patients
        else:
            remaining_size -= len(dt_design_5_step2up)

        print("remaining size for Round " + str(round) + ": " + str(remaining_size))

        ## determine whether move on to step 2:
        if len(highest_cluster) == 1: # if there is only one plan left

            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size) # record the last round sample size
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(1)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        ## sample size determination:
        if round == 2:
            dt_design_5_step2up_overall = dt_design_5
        orr_1 = dt_design_5_step2up_overall['response'].mean() # observed overall response rates for previous rounds
        orr_2 = orr_1 + delta
        n_1 = len(dt_design_5_step2up_overall)

        size_step2up = sample_size_calc(orr_1, n_1, delta=delta, alpha=alpha, power=power) # total size for dataset
        if size_step2up > 0 and size_step2up < 1000:
            size_step2up = 1000 # if size in [0,1000], then it is 1000 for this round.
        elif size_step2up >= 1000:
            size_step2up = min(size_step2up, int(total_n/n_rounds)) # dataset size capped by the n_patient_per_plan
        else:
        # if size_step2up <= 0:
            # the process stops at this step
            print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up) + 'lt 0, break')
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)
            for r in range(round+1, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                      n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_step2up['recruitment_plan'].unique()
            if round == 2:
                supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)] # data from step 1
            else:
                # step from previous rounds
                supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]

            dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(0)
            stopping_dict['early_stopping_size'].append(1)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_step2up_overall['plan_response_rate']))

            for r in range(round+1, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare for chance of better performance:
            temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)
            break

        print('Calculated size for Round ' + str(round) + ': ' + str(size_step2up))

        sample_size_dict["step"+str(round)+"_sample_size"].append(size_step2up)

        ## save benchmark results for last round:
        dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                            n_patient_per_plan = n_patient_per_plan, seed=seed+round)
        maxidx = np.argmax(dt_benchmark['plan_response_rate'])
        random_rr_dict["step"+str(round)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
        random_rr_dict["step"+str(round)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

        ## data generation, combine previous data:
        dt_design_5_step2up = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next,round=round,
                                                  design_number = n_design, n_rounds = n_rounds,
                                                  n_patient_per_plan = n_patient_per_plan, size = int(size_step2up), seed=seed)

        plan_list = dt_design_5_step2up['recruitment_plan'].unique()
        if round == 2:
            supp = dt_design_5[dt_design_5['recruitment_plan'].isin(plan_list)]
        else:
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
        dt_design_5_step2up_overall = pd.concat([dt_design_5_step2up, supp.reset_index(drop=True)], axis = 0)
        dt_design_5_step2up_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

        ## Ensemble model fitting:
        dt_design_5_step2up.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)
        y_pred2up = ensemble_model_fit(data = dt_design_5_step2up_overall, data_pred = dt_design_5_step2up)
        pred_df_step2up = pd.DataFrame(np.hstack((dt_design_5_step2up,  y_pred2up[:, 1].reshape(-1, 1))),
                                    columns=list(dt_design_5_step2up.columns) + ['predicted_response_rate'])
        pred_df_rr_step2up = pred_df_step2up.groupby([f'Design_Feature_{i+1}' for i in range(n_design)]+['recruitment_plan'])['predicted_response_rate'].mean().reset_index(name='predicted_response_rate')

        ## select recruitment plans:
        x = pred_df_rr_step2up['predicted_response_rate'].values

        if len(x) <= 10:
            best_k = 2
        else:
            best_k = kmeans_fit(data = x)['best_k']
        kmeans_results = kmeans_bhattacharyya(data=x, k=best_k)

        ## match the cluster results back to the original data
        highest_cluster_previous = highest_cluster # save previous cluster results

        merged_df = pd.merge(pred_df_rr_step2up, kmeans_results['clusters'][['predicted_response_rate', 'cluster_number']], on='predicted_response_rate')
        cluster_with_highest_rate = kmeans_results['clusters'].groupby('cluster_number')['predicted_response_rate'].mean().idxmax()
        highest_cluster = merged_df[merged_df['cluster_number']==cluster_with_highest_rate].reset_index(drop=True)
        highest_cluster.sort_values(by='predicted_response_rate', ascending=False, inplace=True)

        # p_vec_previous = p_vec_next # save p_vec of previous round
        p_vec_next = np.array(highest_cluster['predicted_response_rate']/np.sum(highest_cluster['predicted_response_rate']))

        highest_cluster['p_vec'] = p_vec_next

        highest_cluster = pd.merge(highest_cluster, dt_design_5_step2up[['recruitment_plan','plan_response_rate']].drop_duplicates(), how='left', on='recruitment_plan')

        ## prepare to chance of better performance:
        temp = dt_design_5_step2up[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
        event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print("step"+str(round), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
        # print(temp)

        ## check early stopping predicted ORR:
        orr_df = pd.merge(highest_cluster_previous, highest_cluster[['recruitment_plan','predicted_response_rate','p_vec']], on='recruitment_plan', how='left')
        orr_df.fillna(0, inplace=True)
        p_orr_1 = np.dot(np.array(orr_df['p_vec_x']), np.array(orr_df['predicted_response_rate_y']))
        p_orr_2 = np.dot(np.array(orr_df['p_vec_y']), np.array(orr_df['predicted_response_rate_y']))
        print("orr termination", p_orr_1, p_orr_2, p_orr_2 - p_orr_1)

        if (p_orr_2 - p_orr_1 < epsilon):
            # step 3 use the same strategy of step2
            print(i, p_orr_1, p_orr_2, "early stop at Round " + str(round))
            rr_dict["step"+str(round+1)+"_rr"].append(np.dot(np.array(highest_cluster['p_vec']), np.array(highest_cluster['plan_response_rate'])))

            ## save benchmark results for last round:
            dt_benchmark = generate_data_step1(design_number = n_design, n_rounds = n_rounds,
                                                n_patient_per_plan = n_patient_per_plan, seed=seed+round)
            maxidx = np.argmax(dt_benchmark['plan_response_rate'])
            random_rr_dict["step"+str(round+1)+'_random_max_rr'].append(dt_benchmark.iloc[maxidx,6])
            random_rr_dict["step"+str(round+1)+'_random_mean_rr'].append(dt_benchmark['plan_response_rate'].mean())

            ### update remaining size:
            remaining_size -= len(dt_design_5_step2up)
            print("early stop, remaining size for Round" + str(round + 1) + ": " + str(remaining_size))
            sample_size_dict["step"+str(round)+"_sample_size"].append(remaining_size)
            last_round_sample_size.append(remaining_size)

            for r in range(round+2, n_rounds+1):
                sample_size_dict["step"+str(r)+"_sample_size"].append(np.nan)

            dt_design_5_rest = generate_data_step2up(highest_cluster=highest_cluster, p_vec = p_vec_next, round=round,
                                                      design_number = n_design, n_rounds = n_rounds,
                                                      n_patient_per_plan = n_patient_per_plan, size = int(remaining_size), seed=seed)
            plan_list = dt_design_5_rest['recruitment_plan'].unique()
            supp = dt_design_5_step2up_overall[dt_design_5_step2up_overall['recruitment_plan'].isin(plan_list)]
            dt_design_5_rest_overall = pd.concat([dt_design_5_rest, supp.reset_index(drop=True)], axis = 0)
            dt_design_5_rest_overall.drop(['predicted_response_rate','cluster_number','p_vec'], axis=1, inplace=True)

            stopping_dict['early_stopping'].append(1)
            stopping_dict['early_stopping_plan'].append(0)
            stopping_dict['early_stopping_orr'].append(1)
            stopping_dict['early_stopping_size'].append(0)

            ## record final plan number
            final_plan_number.append(len(p_vec_next))

            ## record highest responserate overall
            highest_rr_overall.append(np.max(dt_design_5_rest_overall['plan_response_rate']))

            for r in range(round+2, n_rounds+1):
                rr_dict["step"+str(r)+"_rr"].append(np.nan)

            ## prepare to chance of better performance:
            temp = dt_design_5_rest[['recruitment_plan','plan_response_rate', 'group_size']].drop_duplicates()
            event_list += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            event_list_adaptive += (np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print("step"+str(round+1), np.dot(temp['plan_response_rate'].values, temp['group_size'].values))
            # print(temp)

            result = binomtest(int(event_list), n=total_n, p=p0)
            # print(int(event_list), total_n, p0)
            orr_total.append(event_list)
            orr_total_adaptive.append(event_list_adaptive)
            better_chance.append(1 if result.pvalue < 0.05 else 0)
            # print(result.pvalue)

            break



1
['step1_rr: 0', 'step2_rr: 0', 'step3_rr: 0', 'step4_rr: 0', 'step5_rr: 0', 'step6_rr: 0', 'step7_rr: 0', 'step8_rr: 0']


ValueError: shapes (64,7) and (9,) not aligned: 7 (dim 1) != 9 (dim 0)

### Results summary

In [ ]:
rr_df = pd.DataFrame(rr_dict)
rr_df

In [ ]:
# Calculate average rounds:
row_non_nan_counts = rr_df.count(axis=1)

print(f"{np.mean(row_non_nan_counts):.1f} ({np.std(row_non_nan_counts):.1f})")

In [ ]:
# orr for each round:
result_dict = {}
for key, values in rr_df.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"

for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
# overall orr:
result_dict = np.array(orr_total) / total_n
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# highest true rr:
print(np.mean(highest_true_rr))
print(np.std(highest_true_rr))

In [ ]:
# adaptive learning orr:
result_dict = np.array(orr_total_adaptive) / (total_n-34976)
mean_result = np.mean(result_dict)
std_result = np.std(result_dict)

print(f"Mean: {mean_result:.3f}, StD: {std_result:.3f}")

In [ ]:
# average plan number for each round:
result_dict = {}
for key, values in plan_number_dict.items():
    mean_value = np.mean(values)
    std_value = np.std(values)
    result_dict[key] = f"{mean_value:.1f} ({std_value:.1f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")

In [ ]:
means = {key: np.mean(values) for key, values in stopping_dict.items()}
means

In [ ]:
print(np.mean(last_round_sample_size), np.std(last_round_sample_size))

In [ ]:
# plan_number_dict = {"step{}_plan_number".format(r): [] for r in range(1, n_rounds + 1)}
result_dict = {}
for key, values in sample_size_dict.items():
    mean_value = np.nanmean(values)
    std_value = np.nanstd(values)
    result_dict[key] = f"{mean_value:.3f} ({std_value:.3f})"
#     result_dict[key + "_std"] = std_value
for key, value in list(result_dict.items())[:12]:
    print(f"{key}: {value}")